#### To test: 
1. Create a folder ../data/luna16/
2. Create a folder ../data/luna16/subset2
    -Under this folder copy one scan for testing (script will process all the scan at this location) 
      1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405.mhd & raw file 
      (Google drive https://drive.google.com/drive/u/1/folders/13wmubTgm-7sh3MxPGxqmVZuoqi0G3ufW
3. Create a folder ../data/luna16/hdf5
    -Under this copy UNET_weights_H2.h5 (download from google drive)

In [1]:
import pandas as pd
import numpy as np
import h5py
import pandas as pd
import argparse
import SimpleITK as sitk
from PIL import Image
import os, glob 
import os, os.path
import tensorflow as tf
import keras

from ipywidgets import interact
import json
import pickle
from datetime import datetime
from tqdm import tqdm, trange

from UNET_utils import *
from UNET_model_def import *
%matplotlib inline

/prj/anaconda/anaconda3/anaconda3_gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# import argparse
# parser = argparse.ArgumentParser(description='Prediction on HOLDOUT subset',add_help=True)
# parser.add_argument("--holdout", type=int, default=0, help="HOLDOUT subset for predictions")
# args = parser.parse_args()
# HOLDOUT = args.holdout

In [2]:
HOLDOUT = 5
# HO_dir = 'HO{}/'.format(HOLDOUT)
# data_dir = '/home/tony/data/luna16/'
# model_wghts = 'hdf5/UNET_weights_modelB_H{}.h5'.format(HOLDOUT)

TOP_DIR = "/prj/qct/dl_engcomp/sandiego_scratch/capstone/dataset/hdffiles"
HO_dir = '/HO{}/'.format(HOLDOUT)
data_dir = TOP_DIR + HO_dir


model_wghts = "{}/UNET_weights_modelB_exp1_lr009_BSexp1_H{}.h5".format(data_dir, HOLDOUT)




In [3]:
TILE_HSIZE = 64
TILE_WSIZE = 64
TILE_DSIZE = 64


In [4]:
def model_create_loadWghts_Model_B():
   
    input_shape=(None, None, None, 1)
    model = unet3D_Model6_Model13(input_shape, use_upsampling=True)

    model.load_weights(model_wghts)

    return model

In [20]:
def model_create_loadWghts_Model_A():
   
    model = unet3D_Model6_Model13((None,None,None,1))

    model.load_weights(model_wghts)

    return model

In [21]:
def model_create_loadWghts():
    
    input_shape=(None, None, None, 1)
    model = create_UNET3D(input_shape, use_upsampling=True)

    model.load_weights(data_dir + model_wghts)

    return model

In [6]:
model = model_create_loadWghts_Model_B() 
model.summary()

3D U-Net Segmentation
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input_Image (InputLayer)        (None, None, None, N 0                                            
__________________________________________________________________________________________________
conv1a (Conv3D)                 (None, None, None, N 896         Input_Image[0][0]                
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, N 128         conv1a[0][0]                     
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, N 0           batch_normalization_1[0][0]      
_______________________________________________________________________________________

In [5]:
def find_mask(model, img):
    
    height, width, depth = img.shape
    
    # Need to do ceiling to get the right prediction mask size
    # This way prediction mask will be a multiple of the tile size.
    # So just need to pad whole image to that multiple.
    # At the end we crop back to the original image size.
    pred_height = int(TILE_HSIZE * np.ceil(1.0*height/TILE_HSIZE))
    pred_width = int(TILE_WSIZE * np.ceil(1.0*width/TILE_WSIZE))
    pred_depth = int(TILE_DSIZE * np.ceil(1.0*depth/TILE_DSIZE))
    
    # Prediction mask is now a multiple of the TILE_SIZE.
    prediction_mask = np.zeros((pred_height, pred_width, pred_depth, 1))
    idxH = 0
    
    for startH_idx in range(0, height, TILE_HSIZE):
        
        stopH_idx = startH_idx + TILE_HSIZE
        if stopH_idx > height:
            stopH_idx = height
            
        idxW = 0
        
        for startW_idx in range(0, width, TILE_WSIZE):
            
            stopW_idx = startW_idx + TILE_WSIZE
            if stopW_idx > width:
                stopW_idx = width
                
            idxD = 0
               
            for startD_idx in range(0, depth, TILE_DSIZE):
    
                stopD_idx = startD_idx + TILE_DSIZE
                if stopD_idx > depth:
                    stopD_idx = depth
            
                snippet = img[startH_idx:stopH_idx, startW_idx:stopW_idx, startD_idx:stopD_idx]
                
                tile = np.zeros([TILE_HSIZE,TILE_WSIZE,TILE_DSIZE])
                
                snippet_width = stopW_idx - startW_idx
                snippet_height = stopH_idx - startH_idx
                snippet_depth = stopD_idx - startD_idx
                
                tile[:snippet_height,:snippet_width,:snippet_depth] = snippet
                
                tile = np.expand_dims(tile, 0)
                tile = np.expand_dims(tile, -1)
                print(tile.shape)

                tile_mask = model.predict(tile, verbose=0)
                
                prediction_mask[idxH:(idxH+TILE_HSIZE), idxW:(idxW+TILE_WSIZE), idxD:(idxD+TILE_DSIZE), :] = tile_mask[0]
                
                idxD += TILE_DSIZE
                
            idxW += TILE_WSIZE
            
        idxH += TILE_HSIZE
    
    
    return prediction_mask[:height, :width, :depth, :]  # Truncate to original image size
    
    

In [6]:
%%time
t0 = datetime.now()
predictions_dict = {}
size_dict = {}
model = model_create_loadWghts_Model_B() 
fileCount = len(glob.glob(data_dir + 'subset5/' + '*.mhd'))
                
for f in tqdm(glob.glob(data_dir + 'subset5/' + '*.mhd'), total=fileCount, unit="files") :
    print ("\n Processing scan file: {}".format(os.path.basename(f)))
    seriesuid = os.path.splitext(os.path.basename(f))[0]
    # Step-1
    itk_img = sitk.ReadImage(f) 
    img_np_array = sitk.GetArrayFromImage(itk_img)
    original_size = img_np_array.shape
    print ("Original-Size of loaded image : {}".format(original_size))
    # Step-2 
    itk_img_norm = normalize_img(itk_img)
    img_np_array_norm = sitk.GetArrayFromImage(itk_img_norm)
    normalized_size = img_np_array_norm.shape
    # Step-3 
    img = img_np_array_norm.copy()
#     img = normalize_HU(img_np_array_norm)
    img = np.swapaxes(img, 0,2)   ##needed as SITK swaps axis  
    print ("Normalized input image size: {}".format(img.shape))
    
    predicted_mask = find_mask(model, img)
    predictions_dict[seriesuid] = (img.shape, img, predicted_mask)
    size_dict[seriesuid] = img.shape
    

print('Predicted Mask sum for entire scan: {}'.format(np.sum(predicted_mask)))
pickle.dump(predictions_dict, open('Model_B_noHU_entire_predictions_{}.dat'.format(seriesuid), 'wb'))
pickle.dump(size_dict, open('Model_B_noHU_entire_size_{}.dat'.format(seriesuid), 'wb'))    
print('Processing runtime: {}'.format(datetime.now() - t0))

3D U-Net Segmentation


  0%|          | 0/1 [00:00<?, ?files/s]


 Processing scan file: 1.3.6.1.4.1.14519.5.2.1.6279.6001.112740418331256326754121315800.mhd
Original-Size of loaded image : (149, 512, 512)
Normalized input image size: (400, 400, 372)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 1)
(1, 64, 64, 64, 

KeyboardInterrupt: 

In [129]:
predicted_mask.shape

(400, 400, 372, 1)

In [130]:
img.shape

(400, 400, 372)

In [8]:
def displaySlice(sliceNo):
    
    plt.figure(figsize=[20,20]);    

    plt.subplot(1,2,1)
    plt.title("Predicted Mask")
    plt.imshow(np.round(predicted_mask[:, sliceNo,:,0]), cmap='bone');
    #plt.imshow(np.round(predicted_mask[100:350,100:350, sliceNo,0]), cmap='bone');
    
    plt.subplot(1,2,2)
    plt.title("Overlay Mask")
    plt.imshow(img[:, :, sliceNo].T, cmap="bone");
    #plt.imshow(img[sliceNo,:,:].T, cmap="bone");
    #plt.imshow(predicted_mask[:,:,sliceNo,0]>0.01, alpha=0.5, cmap='Reds');
    plt.imshow(predicted_mask[:, sliceNo,:,0], alpha=0.5, cmap='Reds');
    
    plt.show()
    
interact(displaySlice, sliceNo=(1,img.shape[2],1)); #172

interactive(children=(IntSlider(value=186, description='sliceNo', max=372, min=1), Output()), _dom_classes=('widget-interact',))

### Test code below

In [6]:
%%time
t0 = datetime.now()
predictions_dict = {}
size_dict = {}
model = model_create_loadWghts_Model_B() 
fileCount = len(glob.glob(data_dir + 'subset5/' + '*.mhd'))
                
for f in tqdm(glob.glob(data_dir + 'subset5/' + '*.mhd'), total=fileCount, unit="files") :
    print ("\n Processing scan file: {}".format(os.path.basename(f)))
    seriesuid = os.path.splitext(os.path.basename(f))[0]
    # Step-1
    itk_img = sitk.ReadImage(f) 
    img_np_array = sitk.GetArrayFromImage(itk_img)
    original_size = img_np_array.shape
    print ("Original-Size of loaded image : {}".format(original_size))
    # Step-2 
    itk_img_norm = normalize_img(itk_img)
    img_np_array_norm = sitk.GetArrayFromImage(itk_img_norm)
    normalized_size = img_np_array_norm.shape
    # Step-3 
    img = img_np_array_norm.copy()
#     img = normalize_HU(img_np_array_norm)
    img = np.swapaxes(img, 0,2)   ##needed as SITK swaps axis  
    print ("Normalized input image size: {}".format(img.shape))

3D U-Net Segmentation


  0%|          | 0/1 [00:00<?, ?files/s]


 Processing scan file: 1.3.6.1.4.1.14519.5.2.1.6279.6001.112740418331256326754121315800.mhd
Original-Size of loaded image : (149, 512, 512)


100%|██████████| 1/1 [00:12<00:00, 12.69s/files]

Normalized input image size: (400, 400, 372)
CPU times: user 1min 21s, sys: 4.83 s, total: 1min 26s
Wall time: 16 s


In [7]:
test_img = img[:, :, 281:282]
print(test_img.shape)

(400, 400, 1)


In [8]:
# test_img1 = np.expand_dims(np.expand_dims(test_img,-1),-1)
# test_img1.shape

# predict_array = model.predict(test_img1)

# prediction_mask1 = np.round(predict_array)

(400, 400, 1, 1, 1)

In [ ]:
predicted_mask = find_mask(model, test_img)

(1, 400, 400, 1, 1)


In [9]:
def find_mask(model, img):
    
    height, width, depth = img.shape
    TILE_HSIZE = height
    TILE_WSIZE = width
    TILE_DSIZE = depth
    
    # Need to do ceiling to get the right prediction mask size
    # This way prediction mask will be a multiple of the tile size.
    # So just need to pad whole image to that multiple.
    # At the end we crop back to the original image size.
    pred_height = int(TILE_HSIZE * np.ceil(1.0*height/TILE_HSIZE))
    pred_width = int(TILE_WSIZE * np.ceil(1.0*width/TILE_WSIZE))
    pred_depth = int(TILE_DSIZE * np.ceil(1.0*depth/TILE_DSIZE))
    
    # Prediction mask is now a multiple of the TILE_SIZE.
    prediction_mask = np.zeros((pred_height, pred_width, pred_depth, 1))
    idxH = 0
    
    for startH_idx in range(0, height, TILE_HSIZE):
        
        stopH_idx = startH_idx + TILE_HSIZE
        if stopH_idx > height:
            stopH_idx = height
            
        idxW = 0
        
        for startW_idx in range(0, width, TILE_WSIZE):
            
            stopW_idx = startW_idx + TILE_WSIZE
            if stopW_idx > width:
                stopW_idx = width
                
            idxD = 0
               
            for startD_idx in range(0, depth, TILE_DSIZE):
    
                stopD_idx = startD_idx + TILE_DSIZE
                if stopD_idx > depth:
                    stopD_idx = depth
            
                snippet = img[startH_idx:stopH_idx, startW_idx:stopW_idx, startD_idx:stopD_idx]
                
                tile = np.zeros([TILE_HSIZE,TILE_WSIZE,TILE_DSIZE])
                
                snippet_width = stopW_idx - startW_idx
                snippet_height = stopH_idx - startH_idx
                snippet_depth = stopD_idx - startD_idx
                
                tile[:snippet_height,:snippet_width,:snippet_depth] = snippet
                
                tile = np.expand_dims(tile, 0)
                tile = np.expand_dims(tile, -1)
                print(tile.shape)

                tile_mask = model.predict(tile, verbose=0)
                
                prediction_mask[idxH:(idxH+TILE_HSIZE), idxW:(idxW+TILE_WSIZE), idxD:(idxD+TILE_DSIZE), :] = tile_mask[0]
                
                idxD += TILE_DSIZE
                
            idxW += TILE_WSIZE
            
        idxH += TILE_HSIZE
    
    
    return prediction_mask[:height, :width, :depth, :]  # Truncate to original image size

In [55]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [11]:
#!pip install ipywidgets

###### Following sections for reference & WIP code snippets -AL

In [11]:
## Multiple tile test....performance hog, so exploiting the GPU for entire slice without compromising predictions 
##and for better performance  -AL

# slices = 16
# predicted_img = np.zeros(padded_size)

# for i in range(368//slices):
#     tile_1 = padded_img[:224, :224, (i*slices) : slices*(i+1)]
#     tile_2 = padded_img[224:, 224:, (i*slices) : slices*(i+1) ] 

In [12]:
# slices = 8
# predicted_mask = np.zeros(PADDED_SIZE)

# for i in range(24//SLICES):
#     tile = padded_img[:, :, (i*SLICES) : SLICES*(i+1)]
#     tile = tile.reshape(tuple([1] + list (tile.shape) + [1]))
# #     print(tile.shape)

#     tile_predictions = model.predict(tile, verbose=2)
#     tile_mask = tile_predictions[0].reshape(448, 448, 8)
    
#     print (tile_mask.shape)
#     predicted_mask[:, :, (i*SLICES) : SLICES*(i+1)] = tile_mask


In [13]:
# slices = 8
# test_slice = padded_img[:, :, :slices]
# print(test_slice.shape)
# model = model_create_loadWghts(test_slice.shape) 
# # slice_predictions = model.predict(test_slice, verbose=2)

In [14]:
# print ("Shape of predicted mask or segmented image : {}".format(predictions_small_img[0].shape))
# print ("Shape of predicted class : {}".format(predictions_small_img[1].shape))
# predictions_small_img[0] [:, 25 : 26, :]

In [15]:
# ## AL - TEST : making an image of size 48,48,48 with random 0 or 1
# ### Case 2 : As a test created an input image of size (1, 48,48,48,1) 
# # with random 0 or 1; this works fine and able to create predictions successfully
# t2 =  np.random.choice(2,(48,48,48))
# t2 = t2.reshape(tuple([1] + list (t2.shape) + [1]))

# print ("Shape of test input image : {}".format(t2.shape))
# predictions = model.predict(t2, verbose=2)

# print ("Shape of predicted mask or segmented image : {}".format(predictions[0].shape))
# print ("Shape of predicted class : {}".format(predictions[1].shape))
# # predictions[0] [:, 25 : 26, :]

In [16]:
# padded_img[225:232, 225:232, 175]
# predicted_mask[225:232, 225:232, 175]